# Experiments with `start_time`

This notebook introduces functionality for simulating user case in which the experiment steps are triggered at a certain point in time.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
from datetime import datetime


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Let's start defining a model to illustrate this functionality, in this case we choose the SPM

In [2]:
model = pybamm.lithium_ion.SPM()

Usually we define an experiment such that each step is triggered when the previous step is completed. For example, in this case we do a 1C discharge for 20 minutes and then a C/3 charge for 10 minutes. The charge step starts after 20 minutes, i.e. once the discharge step is finished.

In [3]:
experiment = pybamm.Experiment(["Discharge at 1C for 20 minutes", "Charge at C/3 for 10 minutes"])
sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1800.0, step=18.0), Output()), _dom_classes=…

However, if we want to represent a realistic user case we might certain experiments to be run at a certain time instead, even if that means cutting short the previous step. In this case we can pass a starting time as a keyword argument in the `pybamm.step.string` method. The `start_time` should be passed as a `datetime.datetime` object.

In [4]:
s = pybamm.step.string

experiment = pybamm.Experiment(
    [
        s("Discharge at 1C for 1 hour", start_time=datetime(1, 1, 1, 8, 0, 0)),
        s("Charge at C/3 for 10 minutes", start_time=datetime(1, 1, 1, 8, 30, 0)),
        s("Discharge at C/2 for 30 minutes", start_time=datetime(1, 1, 1, 9, 0, 0)),
        s("Rest for 1 hour"),
    ]
)
sim = pybamm.Simulation(model, experiment=experiment)
sim.solve()
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=2.5, step=0.025), Output()), _dom_classes=('…

In the example above, we note that the first step (1C discharge) is cut short as the second step (C/3 charge) start time occurs before the end of the first step. On the other hand, an additional resting period is added after the second step as the third step (C/2 discharge) start time is 20 minutes later than the end of the second step. The final step does not have a start time so it is triggered immediately after the previous step. Note that if the argument `start_time` is used in an experiment, the first step should always have a `start_time`, otherwise the solver will throw an error.

Note that you can use the `datetime.strptime` (see [the docs](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime) for more info) function to convert a string to a datetime object. For example, to start the experiment at 8:30 on the 2nd of January 2023, you can use

In [5]:
datetime.strptime("2023-01-02 8:30:00", "%Y-%m-%d %H:%M:%S")

datetime.datetime(2023, 1, 2, 8, 30)

In [6]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[3] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with electrolyte. Journal of The Electrochemical Society, 166(15):A3693–A3706, 2019. doi:10.1149/2.0341915jes.
[4] Peyman Mohtat, Suhak Lee, Jason B Siegel, and Anna G Stefanopoulou. Towards better estimability of electrode-specific state of health: decoding the cell expansion. Journal of Power Sources, 427:101–111, 201